# Orbit model of P.A. for slit rotation

We want to place the IGRINS slit along the line connecting the two components.  Here we follow the [astrometric orbit fitting tutorial](https://docs.exoplanet.codes/en/stable/tutorials/astrometric/) in the [exoplanet](https://docs.exoplanet.codes/en/stable/) framework to determine the P.A.


In [ ]:
from astropy.time import Time
import numpy as np
import matplotlib.pyplot as plt
import astropy.units as u

%matplotlib inline
%config InlineBackend.figure_format = 'retina'

In [ ]:
import exoplanet as xo

from astropy import constants

# conversion constant from au to R_sun
au_to_R_sun = (constants.au / constants.R_sun).value

We take orbital parameters for Luhman 16 from Table 5 of [Lazorenko & Sahlmann 2018](https://ui.adsabs.harvard.edu/abs/2018A%26A...618A.111L/abstract):

| Param | Value|
|-- | -- |
|$a$ (mas)			| 1784.0|
|$a$ (AU)			| 3.557|
|$e$			      | 0.343 |
|$P$ (yr)			| 27.54|
|$T_0$ (yr)			| 2017.78|
|$i$ (deg)			| 100.26|
|$\omega$ (deg)			| 128.1|
|$\Omega$ (deg)			| 139.67|
|$M_{tot}$ ($M_{\mathrm{Jup} }$)	| 		62.06|
|$M_A$ ($M_{\mathrm{Jup} }$)		| 	33.51|
|$M_B$ ($M_{\mathrm{Jup} }$)		| 	28.55|

In [ ]:
parallax = 501.557 # mas

In [ ]:
a = (3.557 * u.AU).to(u.Rsun).value
e = 0.343
i = 100.26 * np.pi / 180  # [rad]
omega = 128.1 * np.pi / 180  # Pourbaix reports omega_2, but we want omega_1
Omega = 139.67 * np.pi / 180
P = 27.54 * 365.25  # days

T0_orig = Time(2017.78, format="decimalyear")
T0_orig.format = "jd"
T0 = T0_orig.value  # [Julian Date]

In [ ]:
# instantiate the orbit
orbit = xo.orbits.KeplerianOrbit(
    a=a, t_periastron=T0, incl=i, ecc=e, omega=omega, Omega=Omega, 
    m_planet=0.027, m_star=0.032, m_planet_units=u.Msun, r_star = 0.10045
)

In [ ]:
import theano
import theano.tensor as tt

In [ ]:
t_arr = ( T0_orig - P/2*u.day) + np.linspace(0*u.day, P*u.day, num=200)  # days

In [ ]:
t = np.linspace(T0 - P/2, T0 + P/2, num=200)  # days
rho, theta = theano.function([], orbit.get_relative_angles(t, parallax))()

In [ ]:
current_jd = Time.now().jd
current_decyear = Time.now().decimalyear

In [ ]:
rho_now, theta_now = theano.function([], orbit.get_relative_angles(current_jd, parallax))()

In [ ]:
Thst_i = Time(2014.64, format="decimalyear").jd
Thst_f = Time(2016.76, format="decimalyear").jd

In [ ]:
t_hst = np.linspace(Thst_i,Thst_f, num=200)  # days
rho_hst, theta_hst = theano.function([], orbit.get_relative_angles(t_hst, parallax))()

In [ ]:
import seaborn as sns

In [ ]:
sns.set_style('darkgrid')
sns.set_context('talk')

In [ ]:
# Plot the orbit
fig, ax = plt.subplots(nrows=1, figsize=(6, 8))

xs = rho * np.cos(theta)  # X is north
ys = rho * np.sin(theta)  # Y is east
plt.scatter([0], [0], marker='*', s=400, c='#2980b9', ec='k')
ax.plot(ys, xs, lw=2, color='#e67e22')



xnow = rho_now * np.cos(theta_now)  # X is north
ynow = rho_now * np.sin(theta_now)  # Y is east

xhst = rho_hst * np.cos(theta_hst)  # X is north
yhst = rho_hst * np.sin(theta_hst)  # Y is east

ax.plot(yhst, xhst, lw=8, color='#34495e', zorder=1)


label = '{:0.2f}'.format(current_decyear)
plt.scatter([ynow], [xnow], marker='s', s=100, c='#2ecc71', ec='k', 
            label = label, zorder=10)

plt.text(ynow-50, xnow-200, label)

plt.ylim(-1500, 2000)
plt.xlim(1000, -1500)
plt.xlabel(r'$\Delta \alpha \cos \delta$ (mas)')
plt.ylabel(r'$\Delta \delta $ (mas)')
plt.title('Luhman 16 for IGRINS DDT')
plt.savefig('../figures/Luhman16_orbit_demo.png', bbox_inches='tight', dpi=300)

What are the separation and PA right now?

In [ ]:
rho_now

In [ ]:
theta_now * 180/np.pi

## Position on the sky

In [ ]:
Dt_arr = t_arr - t_arr[0]  # days

In [ ]:
Dt_arr.to(u.year)

In [ ]:
compA_coords = theano.function([], orbit.get_star_position(t, parallax))()
compB_coords = theano.function([], orbit.get_planet_position(t, parallax))()

In [ ]:
xA, yA, zA = compA_coords
xB, yB, zB = compB_coords

In [ ]:
plt.plot(xA, yA, label='A')
plt.plot(xB, yB, label='B')
plt.legend();

Let's verify the orientation and enter that info in the DDT observing note so the observers can label each component accurately.